# 为整数对象添加迭代功能

## 一切皆为对象

Python中的一切都是对象：的整数、字符串、函数、类、模块等等。每个对象都具有如下三个要素：

* ID：对象的ID可以通过`id(obj)`获得，在CPython中它返回的是对象在内存中的地址。
* 类型：对象的类型可以通过`type(obj)`获得，它返回表示`obj`类型的对象。
* 内容：每个对象都可以保存种类的内容。

在下面的例子中，`s`表示一个字符串对象，它的类型是`str`。`str`类型本身也是对象，它的类型是`type`类型，而`type`类型也是对象，它的类型是它本身。

In [1]:
s = "abc"
print("id(s):", id(s))
print("type(s):", type(s))
print("type(str):", type(str))
print("type(type):", type(type))

id(s): 2234481678464
type(s): <class 'str'>
type(str): <class 'type'>
type(type): <class 'type'>


## 一切皆是结构体

CPython中的每种类型的对象都与C语言的一种结构体类型对应，例如`int`类型的对象对应`PyLongObject`结构体，`tuple`类型的对象对应`PyTupleObject`结构体，而`type`类型的对象对应`PyTypeObject`结构体。无论是何种类型的结构体，它的头两个字段是固定的：

```c
typedef struct _object {
    Py_ssize_t ob_refcnt;
    struct _typeobject *ob_type;
} PyObject;
```

其中`ob_refcnt`字段保存该对象的引用次数，当引用次数变为0时该对象会被释放。`ob_type`是一个指向`PyTypeObject`结构体的指针，指向该对象所属的类型对象。

下面使用`sys.getrefcount()`获取对象的引用计数：

In [2]:
import sys
print(sys.getrefcount(s))

41


下面用`ffi.cast()`将对象的地址转换为`size_t *`类型的指针，并获取其中的值，也就是`ob_refcnt`字段的值。由于需要把对象传递给`getrefcount()`函数，因此得到的引用计数比调用函数之前要多一个：

In [3]:
import cffi
ffi = cffi.FFI()

ffi.cast("size_t *", id(s))[0]

40

下面通过新的变量名增加一个引用：

In [4]:
s2 = s
ffi.cast("size_t *", id(s))[0]

41

下面通过列表创建10个引用：

In [5]:
slist = [s for i in range(10)]
ffi.cast("size_t *", id(s))[0]

51

`s`的地址加上一个`size_t`字段的大小就得到了第二个字段`obj_type`的地址，它保存的是`s`对应的类型对象的地址，因此它应当与`id(str)`相同。

In [6]:
str_addr = ffi.cast("size_t *", id(s) + ffi.sizeof("size_t"))[0]
print(str_addr, id(str))

1794575248 1794575248


在后面的章节中我们经常会需要把地址转换为保存在该地址的对象，可以通过`ctypes.cast()`实现：

In [7]:
import ctypes
ctypes.cast(str_addr, ctypes.py_object).value

str

## 为整数对象添加迭代功能

当通过`for`语句循环时，Python会尝试调用`iter()`从对象获取迭代对象。`iter()`函数在`bltinmodule.c`中的`builtin_iter()`中定义，其中调用API函数`PyObject_GetIter()`。其代码如下：

```c
PyObject *
PyObject_GetIter(PyObject *o)
{
    PyTypeObject *t = o->ob_type;
    getiterfunc f = NULL;
    f = t->tp_iter;
    if (f == NULL) {
        if (PySequence_Check(o))
            return PySeqIter_New(o);
        return type_error("'%.200s' object is not iterable", o);
    }
    else {
        PyObject *res = (*f)(o);
        if (res != NULL && !PyIter_Check(res)) {
            PyErr_Format(PyExc_TypeError,
                         "iter() returned non-iterator "
                         "of type '%.100s'",
                         res->ob_type->tp_name);
            Py_DECREF(res);
            res = NULL;
        }
        return res;
    }
}
```

上面的代码中，首先获取对象的类型对象，然后访问其中的`tp_iter`字段，如果该字段不为空，则返回该字段对应的函数的返回值。如果希望在Python中修改该字段，则需要知道它的偏移地址。下面通过`cffi`调用C语言的`offsetof()`宏获取`PyTypeObject`结构体中`tp_iter`字段的偏移地址：

In [1]:
import cffi
ffi = cffi.FFI()
ffi.cdef("""
size_t tp_iter_offset;
""")

lib = ffi.verify("""
size_t tp_iter_offset = offsetof(PyTypeObject, tp_iter);
""")

上面的代码需要调用C语言编译器，编译之后就可以通过`lib`获得`tp_iter_offset`的值了，在64位环境下该字段的偏移量为216：

In [2]:
lib.tp_iter_offset

216

我们需要将一个C语言的函数地址写入该字段，这就需要在Python中定义C语言可以调用的函数。使用`ctypes`的`CFUNCTYPE`可以定义函数类型，并使用它将Python函数包装为C语言可以调用的函数`cint_iter()`：

In [3]:
import ctypes
from ctypes import CFUNCTYPE, c_ssize_t, c_void_p

ctypes.pythonapi.Py_IncRef.argtypes = [ctypes.c_size_t]

def int_iter(obj_addr):
    obj = ctypes.cast(obj_addr, ctypes.py_object).value
    iter_obj = iter(range(obj))
    ctypes.pythonapi.Py_IncRef(id(iter_obj))
    return id(iter_obj)

ITER_FUNC = CFUNCTYPE(c_ssize_t, c_ssize_t)
cint_iter = ITER_FUNC(int_iter)

C语言的函数的参数和返回值都是指针，因此这里使用`c_size_t`声明其类型为可以表示指针的整数类型。`obj_addr`参数是调用`iter(obj)`时`obj`对象的地址。首先调用`ctypes.cast()`将地址转换为`py_object`类型，并获取其`value`属性，得到地址中保存的对象。然后调用`iter(range(obj))`创建一个迭代对象。在返回该对象的地址之前，调用`ctypes.pythonapi.Py_IncRef()`将其的引用计数增加1，保证其不被垃圾回收。注意这里不能直接返回`range(obj)`对象，因为在`PyObject_GetIter()`中会调用`PyIter_Check()`检查返回对象是否为迭代对象。

下面试试`int_iter()`函数，它返回的`addr`是迭代对象的地址，由于Python对象对应的结构体中第一个字段保存该对象的引用计数，因此可以将其`cast()`为`size_t*`指针，查看其引用计数值：

In [4]:
addr = cint_iter(id(3))
ffi.cast("size_t *", addr)[0]

1

下面将`cint_inter`函数的地址写入`int`类型对应的结构体的`tp_iter`字段中：

In [5]:
tp_iter_pointer = ffi.cast("size_t *", id(int) + lib.tp_iter_offset)
tp_iter_pointer[0] = ctypes.cast(cint_iter, c_void_p).value

`tp_iter`字段中有了正确的函数地址，就可以对整数进行迭代了：

In [6]:
for i in 5:
    print(i)

0
1
2
3
4


最后检查一下通过`iter()`得到的迭代对象的引用计数是否正确，由于`sys.getrefcount()`本身也会增加引用计数，因此迭代对象有两次引用：

In [7]:
import sys
i = iter(10)
sys.getrefcount(i)

2

## 小结

下面小结本章用到的各种函数：

* 当需要调用C语言宏时，最简单的方法是使用`cffi.FFI`对象的`verify()`，它将对其中的C语言程序编译，生成扩展模块，并根据`cdef()`方法的参数生成Python的调用接口。

* 可以通过`ffi.cast('数据类型 *', addr)[0]`获取地址`addr`中保存的值。

* 使用`ctypes`可以将Python函数包装为C语言可调用的函数，注意这些函数的参数和返回值类型都是C语言的类型。

* 可以通过`ctypes.pythonapi`调用Python的C语言API函数，注意有时需要通过`argtypes`和`restype`设置函数参数类型和返回值类型。

* `ctypes.cast(obj_addr, ctypes.py_object).value`可以通过对象的地址得到对象的引用。